In [ ]:
%%time
!python dataset_creation.py

--- 3D Fossil Slice Dataset Generator v9 (verbose) ---
[CONFIG] Output Dir: 3d_fossil_dataset_clean_v10
[CONFIG] Target Slices/Species: 10000
[CONFIG] Workers: 16, Format: png

[INFO] Found 79 total 3D models with valid labels.
[INFO] Split complete: 63 models for training, 16 for testing.

── Generating TRAIN set (63 models) ──
  [QUOTA] Estimating maximum achievable slices per species...

  Processing model 1/63: avizo_Baculogypsain_test_new_stem_regular_thickness_4_Baculogypsina_4_test_Recon_May_5_2025.nii ➜ 'Baculogypsina'
    [LOAD] Reading & resampling volume...
    [PROC] Axis-aligned views (jitter=True)...
    [PROC] Heavy rotated views (parallel)...
    [SAVE] Writing 497 slices...

  Processing model 2/63: avizo_Fallotia_756_a_stainless_steel_specimen_3_Fallotia_756_a_specimen_3_Recon_May_1_2025.nii ➜ 'Fallotia'
    [LOAD] Reading & resampling volume...
    [PROC] Axis-aligned views (jitter=True)...
    [PROC] Heavy rotated views (parallel)...
    [SAVE] Writing 987 slices...

In [ ]:
# ─────────────────────────────────────────────────────────────
# count_slices_per_species.ipynb  (1 cell – run it!)
# ─────────────────────────────────────────────────────────────
import os, pandas as pd, numpy as np

DATA_DIR = "3d_fossil_dataset_clean"           # adjust if you renamed it
SUFFIXES  = (".png", ".npy")                       # what a slice looks like

rows = []
for split in ("train", "test"):
    root = os.path.join(DATA_DIR, split)
    if not os.path.isdir(root):
        print(f"[WARN] Split folder missing: {root}")
        continue
    for species in os.listdir(root):
        sp_dir = os.path.join(root, species)
        if not os.path.isdir(sp_dir): continue
        n = sum(1 for f in os.listdir(sp_dir)
                if f.lower().endswith(SUFFIXES))
        rows.append({"species": species, "split": split, "n_slices": n})

df = pd.DataFrame(rows)
pivot = df.pivot(index="species", columns="split", values="n_slices").fillna(0).astype(int)
display(pivot)                       # nice spreadsheet in Jupyter

# ---------- quick balance diagnostics ----------
print("\n── TRAIN BALANCE ──")
train_counts = pivot["train"]
print(train_counts.describe().to_string())         # min/mean/max/stdev
print(f"Coefficient of variation (CV): {train_counts.std()/train_counts.mean():.2%}")

print("\n── TEST BALANCE ──")
test_counts  = pivot["test"]
print(test_counts.describe().to_string())
print(f"Coefficient of variation (CV): {test_counts.std()/test_counts.mean():.2%}")


split,test,train
species,,
Alveolina,5129,5259
Arumella,2163,4740
Ataxophragmium,4898,4936
Baculogypsina,2980,3240
Chrysalidina,5111,4861
Coskinolina,5060,4933
Elphidiella,3385,5060
Fallotia,5009,5089
Lockhartia,5359,5133



── TRAIN BALANCE ──
count      12.000000
mean     4845.750000
std       530.339707
min      3240.000000
25%      4840.500000
50%      4934.500000
75%      5100.000000
max      5259.000000
Coefficient of variation (CV): 10.94%

── TEST BALANCE ──
count      12.000000
mean     4289.000000
std      1060.040651
min      2163.000000
25%      3514.000000
50%      4953.500000
75%      5072.750000
max      5359.000000
Coefficient of variation (CV): 24.72%


## Creation of the 3-sets / train / val / test  with low Coefficient of variation (CV)

In [ ]:
# =====================================================================
#  💾  ONE-SHOT  TRAIN / VAL  DISK  SPLIT  ——  exact-target + safe-cap
# =====================================================================
import os, re, shutil, pathlib, random, collections
from itertools import combinations

# ---------------------------- user knobs -----------------------------
DATA_ROOT         = pathlib.Path("3d_fossil_dataset_clean")
SRC_TRAIN_DIR     = DATA_ROOT / "train"
DST_TRAIN_DIR     = DATA_ROOT / "train_split"
DST_VAL_DIR       = DATA_ROOT / "val_split"

TARGET_VAL_SLICES = 1000     # soft target
VAL_CAP           = 1200     # hard cap (unless impossible)
SEED              = 42
USE_SYMLINKS      = False     # True → symlink • False → hard-/copy
# ---------------------------------------------------------------------

random.seed(SEED)

for d in (DST_TRAIN_DIR, DST_VAL_DIR):
    if d.exists():
        shutil.rmtree(d)

# ---------- gather slices by species → modelID -----------------------
pngs = sorted((SRC_TRAIN_DIR).glob('*/*.png'))
assert pngs, f"No PNGs in {SRC_TRAIN_DIR}"
by_sp = collections.defaultdict(lambda: collections.defaultdict(list))
for p in pngs:
    sp  = p.parent.name
    mid = re.sub(r"_s\d+\.png$", "", p.name)
    by_sp[sp][mid].append(p)

train_files, val_files = [], []

# ---------------- per-species split ----------------------------------
for sp, model_dict in by_sp.items():
    models   = list(model_dict.items())              # [(mid, [paths]), …]
    n_models = len(models)
    sizes    = {mid: len(s) for mid, s in models}

    # ---- 1) brute-force to hit TARGET as closely as possible ---------
    best, best_diff = None, float('inf')
    for k in range(1, n_models + 1):
        for combo in combinations(models, k):
            tot = sum(len(s) for _, s in combo)
            diff = abs(tot - TARGET_VAL_SLICES)
            if diff < best_diff:
                best, best_diff = combo, diff
                if diff == 0:
                    break
        if best_diff == 0:
            break
    if best is None:                               # degenerate safety
        best = [min(models, key=lambda kv: len(kv[1]))]

    val_set   = {mid for mid, _ in best}
    val_total = sum(sizes[mid] for mid in val_set)

    # ---- 2) cap enforcement while keeping ≥1 model -------------------
    if val_total > VAL_CAP:
        # sort val models big→small; pop while >cap AND >1 model left
        for mid in sorted(val_set, key=lambda m: sizes[m], reverse=True):
            if val_total <= VAL_CAP or len(val_set) == 1:
                break
            val_set.remove(mid)
            val_total -= sizes[mid]

    # still over cap? that means even smallest model > cap → keep it.
    # (No further action; we accept the overshoot.)

    train_set = {mid for mid, _ in models if mid not in val_set}

    # ---- 3) book-keeping --------------------------------------------
    for mid, paths in model_dict.items():
        (val_files if mid in val_set else train_files).extend(paths)

    tot_sp = sum(sizes.values())
    print(f"{sp:12s} | models {n_models:2d} "
          f"→ val {len(val_set):2d} ({val_total}/{tot_sp}) "
          f"train {len(train_set):2d}")

# ---------------- helper to write files ------------------------------
def _dump(src_list, dst_root):
    for src in src_list:
        dst = dst_root / src.relative_to(SRC_TRAIN_DIR)
        dst.parent.mkdir(parents=True, exist_ok=True)
        try:
            if USE_SYMLINKS:
                os.symlink(src, dst)
            else:
                try:    os.link(src, dst)
                except OSError:
                    shutil.copy2(src, dst)
        except FileExistsError:
            pass

# ---------------- materialise to disk --------------------------------
print(f"\n→ writing {len(train_files):,} train images …")
_dump(train_files, DST_TRAIN_DIR)

print(f"→ writing {len(val_files):,}   val images …")
_dump(val_files,   DST_VAL_DIR)

print("\n✓ DONE — new folders ready:",
      f"\n   {DST_TRAIN_DIR}\n   {DST_VAL_DIR}")



##### Alveolina    | models  3 → val  1 (1724/5259) train  2
##### Arumella     | models  4 → val  1 (1143/4740) train  3
##### Ataxophragmium | models  5 → val  1 (993/4936) train  4
##### Baculogypsina | models  6 → val  2 (997/3240) train  4
##### Chrysalidina | models 13 → val  3 (1079/4861) train 10
##### Coskinolina  | models  3 → val  1 (1583/4933) train  2
##### Elphidiella  | models  5 → val  1 (1012/5060) train  4
##### Fallotia     | models  5 → val  1 (987/5089) train  4
##### Lockhartia   | models  4 → val  1 (1234/5133) train  3
##### Minoxia      | models  5 → val  1 (1034/5190) train  4
##### Orbitoides   | models  4 → val  1 (1074/4779) train  3
##### Rhapydionina | models  4 → val  1 (1186/4929) train  3

##### → writing 44103  train images …
##### → writing 14046    val images …

##### ✓ DONE — new folders ready: 
#####    3d_fossil_dataset_clean/train_split
#####    3d_fossil_dataset_clean/val_split

In [ ]:
# ─────────────────────────────────────────────────────────────
# count_slices_per_species (train / val / test edition)
# ─────────────────────────────────────────────────────────────
import os, pandas as pd, numpy as np

DATA_DIR  = "3d_fossil_dataset_clean"          # adjust if you renamed it
SPLITS    = ("train_split", "val_split", "test")   # ← NEW
SUFFIXES  = (".png", ".npy")                       # what a slice looks like

rows = []
for split in SPLITS:
    root = os.path.join(DATA_DIR, split)
    if not os.path.isdir(root):
        print(f"[WARN] split folder missing: {root}")
        continue

    for species in os.listdir(root):
        sp_dir = os.path.join(root, species)
        if not os.path.isdir(sp_dir):
            continue

        n = sum(
            1
            for fname in os.listdir(sp_dir)
            if fname.lower().endswith(SUFFIXES)
        )
        rows.append({"species": species, "split": split, "n_slices": n})

# --- tidy up as a pivot table (species × split) ---
df = pd.DataFrame(rows)
pivot = (
    df.pivot(index="species", columns="split", values="n_slices")
      .fillna(0)
      .astype(int)
      .sort_index(axis=1)             # ensure columns appear in order
)
display(pivot)                        # interactive spreadsheet in Jupyter

# ---------- quick balance diagnostics ----------
def diag(col, name):
    print(f"\n── {name.upper()} BALANCE ──")
    print(col.describe().to_string())                # min / mean / max / stdev etc.
    if col.mean() > 0:
        cv = col.std() / col.mean()
        print(f"Coefficient of variation (CV): {cv:.2%}")
    else:
        print("No samples in this split.")

for split in SPLITS:
    if split in pivot:
        diag(pivot[split], split.replace("_split", ""))


split,test,train_split,val_split
species,,,
Alveolina,5129,3535,1724
Arumella,2163,3597,1143
Ataxophragmium,4898,3943,993
Baculogypsina,2980,2243,997
Chrysalidina,5111,3782,1079
Coskinolina,5060,3350,1583
Elphidiella,3385,4048,1012
Fallotia,5009,4102,987
Lockhartia,5359,3899,1234



── TRAIN BALANCE ──
count      12.000000
mean     3675.250000
std       511.123918
min      2243.000000
25%      3581.500000
50%      3762.500000
75%      3969.250000
max      4156.000000
Coefficient of variation (CV): 13.91%

── VAL BALANCE ──
count      12.00000
mean     1170.50000
std       240.88228
min       987.00000
25%      1008.25000
50%      1076.50000
75%      1198.00000
max      1724.00000
Coefficient of variation (CV): 20.58%

── TEST BALANCE ──
count      12.000000
mean     4289.000000
std      1060.040651
min      2163.000000
25%      3514.000000
50%      4953.500000
75%      5072.750000
max      5359.000000
Coefficient of variation (CV): 24.72%


## Creation of Segmented images with black background 

In [ ]:
!python run_full_segmentation_black_bg.py